<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 13: Alexis Cornby makes her living buying and selling corn.
On January 1, she has 50 tons of corn and \$1,000. On the
first day of each month Alexis can buy corn at the following
prices per ton: January, \$300; February, \$350; March, \$400;
April, \$500. On the last day of each month, Alexis can sell
corn at the following prices per ton: January, \$250; February,
\$400; March, \$350; April, \$550. Alexis stores her corn in a
warehouse that can hold at most 100 tons of corn. She must
be able to pay cash for all corn at the time of purchase. Use
linear programming to determine how Alexis can maximize
her cash on hand at the end of April.

In [2]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 48.2 MB/s eta 0:00:00


In [3]:
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

# --- Data --- #
months = [1,2,3,4]
buy_price  = {1:300, 2:350, 3:400, 4:500}
sell_price = {1:250, 2:400, 3:350, 4:550}
warehouse_capacity = 100
I0 = 50
C0 = 1000

# --- Model --- #
m = LpProblem("Alexis_Corn_Trading", LpMaximize)

# Decision variables #
B = {t: LpVariable(f"B_{t}", lowBound=0) for t in months}  # buy
S = {t: LpVariable(f"S_{t}", lowBound=0) for t in months}  # sell
I = {t: LpVariable(f"I_{t}", lowBound=0, upBound=warehouse_capacity) for t in months}  # inventory
C = {t: LpVariable(f"C_{t}", lowBound=0) for t in months}  # cash

# Inventory balance #
for t in months:
    if t == 1:
        m += I[t] == I0 + B[t] - S[t]
    else:
        m += I[t] == I[t-1] + B[t] - S[t]

# Cash balance #
for t in months:
    if t == 1:
        m += C[t] == C0 - B[t]*buy_price[t] + S[t]*sell_price[t]
    else:
        m += C[t] == C[t-1] - B[t]*buy_price[t] + S[t]*sell_price[t]

# Cash available for purchases #
for t in months:
    if t == 1:
        m += B[t]*buy_price[t] <= C0
    else:
        m += B[t]*buy_price[t] <= C[t-1]

# Objective: maximize cash at end of April #
m += C[4]

# Solve #
m.solve()

# Results #
print("Optimal cash at end of April: $", value(C[4]))
for t in months:
    print(f"Month {t}: Buy={B[t].value():.1f} tons, Sell={S[t].value():.1f} tons, Inventory={I[t].value():.1f} tons, Cash=${C[t].value():.2f}")


Optimal cash at end of April: $ 29333.333
Month 1: Buy=3.3 tons, Sell=0.0 tons, Inventory=53.3 tons, Cash=$0.00
Month 2: Buy=0.0 tons, Sell=0.0 tons, Inventory=53.3 tons, Cash=$0.00
Month 3: Buy=0.0 tons, Sell=0.0 tons, Inventory=53.3 tons, Cash=$0.00
Month 4: Buy=0.0 tons, Sell=53.3 tons, Inventory=0.0 tons, Cash=$29333.33
